# Colab-traiNNer

victorca25's BasicSR fork: [victorca25/traiNNer](https://github.com/victorca25/traiNNer)

My fork: [styler00dollar/Colab-traiNNer](https://github.com/styler00dollar/Colab-traiNNer)

In [ ]:
#@title Check GPU

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print("GPU: " + gpu[1])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install

In [ ]:
#@title Install dependencies
%cd /content/

# create empty folders
!mkdir /content/hr
!mkdir /content/lr
!mkdir /content/val_hr
!mkdir /content/val_lr
 
!mkdir /content/masks
!mkdir /content/validation
!mkdir /content/data
!mkdir /content/logs/

!git clone https://github.com/styler00dollar/Colab-traiNNer
!pip install nebulgym basicsr albumentations wget tfrecord x-transformers adamp efficientnet_pytorch tensorboardX vit-pytorch swin-transformer-pytorch madgrad timm pillow-avif-plugin kornia omegaconf \
  git+https://github.com/styler00dollar/piq.git git+https://github.com/vballoli/nfnets-pytorch git+https://github.com/PyTorchLightning/pytorch-lightning.git --force-reinstall -U
!pip3 install --pre torch torchvision torchaudio torchtext --extra-index-url https://download.pytorch.org/whl/nightly/cu116 --force-reinstall -U

## Optional

In [ ]:
#@title (optional) download precompiled mmcv (for GLEAN)
%cd /content/
!pip uninstall mmcv -y
!pip uninstall mmcv-full -y
!gdown --id 1--PoTPGKwAqGJsmaLYSEiqJy1yWMTx0G
!pip install mmcv_full-1.3.5-cp37-cp37m-linux_x86_64.whl

In [ ]:
#@title (optional) compiling and installing mmcv (for GLEAN)
!pip install torch torchvision torchaudio -U
!pip uninstall mmcv -y
!pip uninstall mmcv-full -y
!pip install mmcv-full

In [ ]:
#@title (optional) ninja (for GFPGAN / GPEN / co-mod-gan)
%cd /content
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 

In [ ]:
#@title (optional) correlation package (for ABME)
%cd /content/
!sudo rm -rf ABME
!git clone https://github.com/JunHeum/ABME
%cd /content/ABME/correlation_package
#!python setup.py install
!python setup.py build install

In [ ]:
#@title (optional) install cupy (for EDSC)
!curl https://colab.chainer.org/install | sh -

In [ ]:
#@title (optional) upgrade pytoch
!pip3 install --pre torch torchvision torchaudio torchtext -f https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html -U --force-reinstall

In [ ]:
#@title (optional) Adam8bit optimizer
!pip install bitsandbytes-cuda111

In [ ]:
#@title (Optional) Install turboJPEG
%cd /content/
!sudo apt-get install -y libturbojpeg 
!pip install PyTurboJPEG

In [ ]:
#@title TPU
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

# Make Directories and Copy Data
You need to upload the data and then extract it within colab. You can use Google Drive for that.

In [ ]:
%cd /content/
!cp "/content/drive/MyDrive/dataset.tar" "/content/data.tar"
!7z x /content/data.tar

In [ ]:
# if dataset is not big, copy it into ram instead
%cd /content/
!mkdir /content/ramdisk
!sudo mount -t tmpfs none /content/ramdisk
!sudo chmod 777 /content/ramdisk/

%cd /content/ramdisk/
!cp "/content/drive/MyDrive/dataset.tar" "/content/ramdisk/data.tar"
!7z x "/content/ramdisk/data.tar"

# Train

In [ ]:
#@title config.yaml
%%writefile /content/Colab-traiNNer/code/config.yaml
name: template
scale: 2
gpus: 1 # amount of gpus, 0 = cpu
distributed_backend: ddp # dp, ddp (for multi-gpu training)
tpu_cores: 8 # 8 if you use a Google Colab TPU
use_tpu: False
use_amp: False
use_swa: False
progress_bar_refresh_rate: 20
default_root_dir: '/content/'

# Dataset options:
datasets:
  train:
    # DS_inpaint: hr is from dataroot_HR, loads masks
    # DS_lrhr: loads lr from dataroot_LR and hr from dataroot_HR
    # DS_video: video dataloader which has 3 frames as input (look into data/data_video.py for more details)
    # DS_inpaint_TF: takes one tfrecord file as dataset input, but the validation is still just green masked images like in DS_inpaint
    # DS_video_direct: direcly copy .npy files into GPU and avoiding CPU processing (upgrade to newest nvidia drivers and cuda, linux only)
    # only works with n_workers = 0, use pipeline_threads instead
    # DS_realesrgan: will use the realesrgan dataloader (only uses hr folder)
    # pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110

    mode: DS_realesrgan # DS_video | DS_video_direct | DS_inpaint_TF | DS_inpaint  | DS_lrhr | DS_realesrgan
    amount_files: 7 # tfrecord files do not store amount of images and are infinite, specify the images inside of it

    tfrecord_path: "/content/tfrecord/tfrecord-r09.tfrecords"
    dataroot_HR: '/content/hr/' # Original, with a single directory. Inpainting will use this directory as source image.
    dataroot_LR: '/content/lr/' # Original, with a single directory
    loading_backend: 'OpenCV' # 'PIL' | 'OpenCV' | 'turboJPEG' # install needed for turboJPEG, turboJPEG only for DS_video, 'PIL' for DS_inpaint_TF

    n_workers: 16 # 0 to disable CPU multithreading, or an integrer representing CPU threads to use for dataloading
    pipeline_threads: 16 # only for DS_video_direct
    batch_size: 15 # 6
    
    # does not apply to video dataloaders, look into python file instead
    HR_size: 128 # The resolution the network will get. Random crop gets applied if that resolution does not match.
    image_channels: 3 # number of channels to load images in

    masks: '/content/masks/' # only for inpainting
    mask_invert_ratio: 0.3 # 0.3 = 30% of masks will be inverted
    max_epochs: 2000
    save_step_frequency: 5000 # also validation frequency

    # if edge data is required
    canny_min: 100
    canny_max: 150

    # OTF downscaling
    # This will downscale the HR image with a randomly chosen filter and ignore the LR folder.
    # otf_filter_probs defines the cumulative (additive) weights used to select a downscaling filter
    # KERNEL will randomly apply one of the kernels generated using
    # https://github.com/victorca25/DLIP/tree/main/kgan
    apply_otf_downscale: False
    otf_filter_types: ['KERNEL', 'NEAREST', 'BILINEAR', 'AREA', 'BICUBIC', 'LANCZOS']
    otf_filter_probs: [0.25, 0.4, 0.55, 0.70, 0.85, 1.0]
    kernel_path: '/content/kernels'

    # Image augmentations (only for lrhr dataloader). Set 'True' to use.
    # To customize individual augmentations, edit aug_config.yaml.
    # Augmentations will apply in random order.
    ColorJitter: False
    RandomGaussianNoise: False
    RandomPoissonNoise: False
    RandomSPNoise: False
    RandomSpeckleNoise: False
    RandomCompression: False
    RandomAverageBlur: False
    RandomBilateralBlur: False
    RandomBoxBlur: False
    RandomGaussianBlur: False
    RandomMedianBlur: False
    RandomMotionBlur: False
    RandomComplexMotionBlur: False
    RandomAnIsoBlur: False
    RandomSincBlur: False
    BayerDitherNoise: False
    FSDitherNoise: False
    FilterMaxRGB: False
    FilterColorBalance: False
    FilterUnsharp: False
    FilterCanny: False
    SimpleQuantize: False
    KMeansQuantize: False
    CLAHE: False
    RandomGamma: False
    Superpixels: False
    RandomCameraNoise: False

  val:
    loading_backend: 'OpenCV' # 'OpenCV' | 'turboJPEG' # install needed for turboJPEG, currently only for DS_video
    dataroot_HR: '/content/val_hr/'
    dataroot_LR: '/content/val_lr/' # Inpainting will use this directory as input

path:
    pretrain_model_G: #"/content/model.pth"
    pretrain_model_G_teacher: 
    pretrain_model_D: 
    checkpoint_path:
    checkpoint_save_path: '/content/'
    validation_output_path: '/content/validation'
    log_path: '/content/logs'

# using a teacher model to generate hr, has same configuration as normal netG
# if teacher is used, same loss functions will be applied to both images
# if you don't use a teacher, set netG to None
network_G_teacher:
    # CEM (for esrgan, not 1x)
    CEM: False # uses hardcoded torch.cuda.FloatTensor

    # comparing feature maps of teacher and student with l1
    l1_feature_maps_weight: 1

    netG: # leave empty if nothing

    #netG: MRRDBNet_FM # RRDB_net (original ESRGAN arch) | MRRDBNet_FM (modified/"new" arch) with feature map knowledge distillation
    #norm_type: null
    #mode: CNA
    #nf: 64 # of discrim filters in the first conv layer (default: 64, good: 32)
    #nb: 3 # (default: 23, good: 8)
    #in_nc: 3 # of input image channels: 3 for RGB and 1 for grayscale
    #out_nc: 3 # of output image channels: 3 for RGB and 1 for grayscale
    #gc: 32
    #convtype: Conv2D # Conv2D | PartialConv2D | doconv | gated | TBC | dynamic | MBConv | CondConv | fft | WSConv
    #nr: 3
    ## for dynamic
    #nof_kernels: 4
    #reduce: 4
    ## RRDB_net
    #net_act: leakyrelu # swish | leakyrelu
    #gaussian: false # true | false # esrgan plus, does not work on TPU because of cuda()
    #plus: false # true | false
    #finalact: None #tanh # Test. Activation function to make outputs fit in [-1, 1] range. Default = None. Coordinate with znorm.
    #upsample_mode: 'upconv'
    #strided_conv: False
    ##group: 1 # unused for now

    # RealESRGAN Anime Compact model (2021)
    #netG: SRVGGNetCompact
    #num_in_ch: 3
    #num_out_ch: 3
    #num_feat: 64
    #num_conv: 16
    #act_type: 'prelu'
    #conv_mode: 3 # 2 | 3
    ## only for rrdb
    #rrdb: False
    #rrdb_blocks: 2
    #convtype: "Conv2D"

# Generator options:
network_G:
    # CEM (for esrgan, not 1x)
    CEM: False # uses hardcoded torch.cuda.FloatTensor
    sigmoid_range_limit: False

    finetune: False # Important for further rfr/dsnet training. Apply that after training for a while. https://github.com/jingyuanli001/RFR-Inpainting/issues/33

    # ESRGAN:
    netG: RRDB_net # RRDB_net (original ESRGAN arch) | MRRDBNet_FM (modified/"new" arch) with feature map knowledge distillation
    norm_type: null
    mode: CNA
    nf: 64 # of discrim filters in the first conv layer (default: 64, good: 32)
    nb: 3 # (default: 23, good: 8)
    in_nc: 3 # of input image channels: 3 for RGB and 1 for grayscale
    out_nc: 3 # of output image channels: 3 for RGB and 1 for grayscale
    gc: 32
    convtype: Conv2D # Conv2D | PartialConv2D | doconv | gated | TBC | dynamic | MBConv | CondConv | fft | WSConv
    nr: 3
    # for dynamic
    nof_kernels: 4
    reduce: 4
    # RRDB_net
    net_act: leakyrelu # swish | leakyrelu
    gaussian: false # true | false # esrgan plus, does not work on TPU because of cuda()
    plus: false # true | false
    finalact: None #tanh # Test. Activation function to make outputs fit in [-1, 1] range. Default = None. Coordinate with znorm.
    upsample_mode: 'upconv'
    strided_conv: False
    #group: 1 # unused for now

    # ASRGAN:
    #which_model_G: asr_resnet # asr_resnet | asr_cnn
    #nf: 64

    # PPON:
    #netG: ppon # | ppon
    ##norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 24
    #in_nc: 3
    #out_nc: 3
    ##gc: 32
    #group: 1
    ##convtype: Conv2D #Conv2D | PartialConv2D

    # SRGAN:
    #netG: sr_resnet # RRDB_net | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 16
    #in_nc: 3
    #out_nc: 3

    # SR:
    #netG: RRDB_net # RRDB_net | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 23
    #in_nc: 3
    #out_nc: 3
    #gc: 32
    #group: 1

    # PAN:
    # netG: pan_net
    # in_nc: 3
    # out_nc: 3
    # nf: 40
    # unf: 24
    # nb: 16
    # self_attention: true
    # double_scpa: false

    # edge-informed-sisr
    #which_model_G: sisr
    #use_spectral_norm: True

    # USRNet
    #netG: USRNet
    #in_nc=4
    #out_nc=3
    #nc=[64, 128, 256, 512]
    #nb=2
    #act_mode='R'
    #downsample_mode='strideconv'
    #upsample_mode='convtranspose'

    # GLEAN (2021)
    # Warning: Does require "pip install mmcv-full"
    #netG: GLEAN
    #in_size: 512
    #out_size: 512
    #img_channels: 4
    #img_channels_out: 3
    #rrdb_channels: 16 # 64
    #num_rrdbs: 8 # 23
    #style_channels: 512 # 512
    #num_mlps: 4 # 8
    #channel_multiplier: 2
    #blur_kernel: [1, 3, 3, 1]
    #lr_mlp: 0.01
    #default_style_mode: 'mix'
    #eval_style_mode: 'single'
    #mix_prob: 0.9
    #pretrained: False # only works with official settings
    #bgr2rgb: False

    # srflow (upscaling factors: 4, 8, 16)
    # Warning: Can be very unstable with batch_size 1, use higher batch_size
    #netG: srflow
    #in_nc: 3
    #out_nc: 3
    #nf: 64
    #nb: 23
    #train_RRDB: false
    #train_RRDB_delay: 0.5
    #flow:
    #  K: 16
    #  L: 3
    #  noInitialInj: true
    #  coupling: CondAffineSeparatedAndCond
    #  additionalFlowNoAffine: 2
    #  split:
    #    enable: true
    #  fea_up0: true
    #  stackRRDB:
    #    blocks: [ 1, 8, 15, 22 ]
    #    concat: true
    #nll_weight: 1
    #freeze_iter: 100000

    # DFDNet
    # Warning: Expects "DictionaryCenter512" in the current folder, you can get the data here: https://drive.google.com/drive/folders/1bayYIUMCSGmoFPyd4Uu2Uwn347RW-vl5
    # Also wants a folder called "landmarks", you can generate that data yourself. Example: https://github.com/styler00dollar/Colab-DFDNet/blob/local/Colab-DFDNet-lightning-train.ipynb
    # Hardcoded resolution: 512px
    #netG: DFDNet
    #dictionary_path: "/content/DictionaryCenter512"
    #landmarkpath: "/content/landmarks"
    #val_landmarkpath: "/content/landmarks"

    # GFPGAN (2021) [EXPERIMENTAL]
    # does require ninja
    # because it compiles files, the startup time is quite long
    #netG: GFPGAN
    #input_channels: 4
    #output_channels: 3
    #out_size: 512
    #num_style_feat: 512
    #channel_multiplier: 1
    #resample_kernel: [1, 3, 3, 1]
    #decoder_load_path: # None
    #fix_decoder: True
    #num_mlp: 8
    #lr_mlp: 0.01
    #input_is_latent: False
    #different_w: False
    #narrow: 1
    #sft_half: False

    # GPEN
    # does require ninja
    # because it compiles files, the startup time is quite long
    # output_channels is hardcoded to 3
    #netG: GPEN
    #input_channels: 4
    #size: 512
    #style_dim: 512
    #n_mlp: 8
    #channel_multiplier: 2
    #blur_kernel: [1, 3, 3, 1]
    #lr_mlp: 0.01
    #pooling: True # Experimental, to have any input size

    # comodgan (2021)
    # needs ninja
    # because it compiles files, the startup time is quite long
    #netG: comodgan
    #dlatent_size: 512
    #num_channels: 3 # amount of channels without mask
    #resolution: 512
    #fmap_base: 16384 # 16 << 10
    #fmap_decay: 1.0
    #fmap_min: 1
    #fmap_max: 512
    #randomize_noise: True
    #architecture: 'skip'
    #nonlinearity: 'lrelu'
    #resample_kernel: [1,3,3,1]
    #fused_modconv: True
    #pix2pix: False
    #dropout_rate: 0.5
    #cond_mod: True
    #style_mod: True
    #noise_injection: True

    # swinir (2021)
    #netG: swinir
    #img_size: 64
    #patch_size: 1
    #upscale: 2
    #in_chans: 3
    #window_size: 8
    #img_range: 1.
    #depths: [6, 6, 6, 6, 6, 6]
    #embed_dim: 180
    #num_heads: [6, 6, 6, 6, 6, 6]
    #mlp_ratio: 2
    #upsampler: 'pixelshuffle'
    #resi_connection: '1conv'

    # swinir2 (2022)
    #netG: swinir2
    #img_size: 56 # 56/60 # 48
    #window_size: 8
    #img_range: 1.
    #depths: [3, 3, 3, 3] # [2, 2, 2, 2]
    #embed_dim: 180 # 24
    #num_heads: [6, 6, 6, 6]
    #mlp_ratio: 2
    #upsampler: 'pixelshuffledirect'
    #use_deformable_block: False
    #first_conv: fft # Conv2D | doconv | TBC | dynamic | fft
    # for dynamic
    #nof_kernels: 4
    #reduce: 4

    # ESRT (2021)
    #netG: ESRT
    #hiddenDim: 32
    #mlpDim: 128

    # RealESRGAN Anime Compact model (2021)
    #netG: SRVGGNetCompact
    #num_in_ch: 3
    #num_out_ch: 3
    #num_feat: 64
    #num_conv: 16
    #act_type: 'prelu'
    #conv_mode: 3 # 2 | 3
    ## only for rrdb
    #rrdb: False
    #rrdb_blocks: 2
    #convtype: "Conv2D"

    # ELAN (2022)
    #netG: elan
    #scale: 4
    #colors: 3
    #window_sizes: [4, 8, 16]
    #m_elan: 24
    #c_elan: 60
    #n_share: 1
    #r_expand: 2
    #rgb_range: 255
    #conv: fft # Conv2D | fft

    # LFT (2022)
    #netG: lft
    #channels: 64
    #angRes: 5
    #layer_num: 4
    #temperature: 10000
    #num_heads: 8
    #dropout: 0.

    # swift (2021)
    #netG: swift
    #in_channels: 3
    #num_channels: 64 
    #num_blocks: 16

    # hat (2022)
    #netG: hat
    #img_size: 64
    #patch_size: 1
    #in_chans: 3
    #embed_dim: 180
    #depths: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
    #num_heads: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
    #window_size: 16
    #compress_ratio: 3
    #squeeze_factor: 30
    #conv_scale: 0.01
    #overlap_ratio: 0.5
    #mlp_ratio: 2
    #qkv_bias: True
    #qk_scale:
    #drop_rate: 0.
    #attn_drop_rate: 0.
    #drop_path_rate: 0.1
    #ape: False
    #patch_norm: True
    #use_checkpoint: False
    #img_range: 1.
    #upsampler: 'pixelshuffle'
    #resi_connection: '1conv'
    #conv: fft # fft | Conv2D

    # RLFN (2022)
    #netG: RLFN
    #in_nc: 3
    #out_nc: 3
    #nf: 46 
    #mf: 48

    # ----Inpainting Generators----
    # DFNet (batch_size: 2+, needs 2^x image input and validation) (2019)
    #netG: DFNet
    #c_img: 3
    #c_mask: 1
    #c_alpha: 3
    #mode: nearest
    #norm: batch
    #act_en: relu
    #act_de: leaky_relu
    #en_ksize: [7, 5, 5, 3, 3, 3, 3, 3]
    #de_ksize: [3, 3, 3, 3, 3, 3, 3, 3]
    #blend_layers: [0, 1, 2, 3, 4, 5]
    #conv_type: normal # partial | normal | deform
    

    # EdgeConnect (2019)
    #netG: EdgeConnect
    #use_spectral_norm: True
    #residual_blocks_edge: 8
    #residual_blocks_inpaint: 8
    #conv_type_edge: 'normal' # normal | partial | deform (has no spectral_norm)
    #conv_type_inpaint: 'normal' # normal | partial | deform

    # CSA (2019)
    #netG: CSA
    #c_img: 3
    #norm: 'instance'
    #act_en: 'leaky_relu'
    #act_de: 'relu'

    # RN (2020)
    #netG: RN
    #input_channels: 3
    #residual_blocks: 8
    #threshold: 0.8

    # deepfillv1 (2018)
    #netG:  deepfillv1

    # deepfillv2 (2019)
    #netG: deepfillv2
    #in_channels:  4
    #out_channels:  3
    #latent_channels:  64
    #pad_type:  'zero'
    #activation:  'lrelu'
    #norm: 'in'
    #conv_type: partial # partial | normal

    # Adaptive (2020)
    #netG: Adaptive
    #in_channels: 3
    #residual_blocks: 1
    #init_weights: True

    # Global (2020)
    #netG: Global
    #input_dim: 5
    #ngf: 32
    #use_cuda: True
    #device_ids: [0]

    # Pluralistic (2019)
    #netG: Pluralistic
    #ngf_E: 32
    #z_nc_E: 128
    #img_f_E: 128
    #layers_E: 5
    #norm_E: 'none'
    #activation_E: 'LeakyReLU'
    #ngf_G: 32
    #z_nc_G: 128
    #img_f_G: 128
    #L_G: 0
    #output_scale_G: 1
    #norm_G: 'instance'
    #activation_G: 'LeakyReLU'

    # crfill (2020)
    #netG: crfill
    #cnum: 48

    # DeepDFNet (experimental)
    #netG: DeepDFNet
    #in_channels:  4
    #out_channels:  3
    #latent_channels:  64
    #pad_type:  'zero'
    #activation:  'lrelu'
    #norm: 'in'

    # partial (2018)
    #netG: partial

    # DMFN (2020)
    #netG: DMFN
    #in_nc: 4
    #out_nc: 3
    #nf: 64
    #n_res: 8
    #norm: 'in'
    #activation: 'relu'

    # pennet (2019)
    #netG: pennet

    # LBAM (2019)
    #netG: LBAM
    #inputChannels: 4
    #outputChannels: 3

    # RFR (use_swa: false, no TPU) (2020)
    #netG: RFR
    #conv_type: partial # partial | deform

    # FRRN (2019)
    #netG: FRRN

    # PRVS (2019)
    #netG: PRVS

    # CRA (HR_size: 512) (2020)
    #netG: CRA
    #activation: 'elu'
    #norm: 'none'

    # atrous (2020)
    #netG: atrous

    # MEDFE (batch_size: 1) (2020)
    #netG: MEDFE

    # AdaFill (2021)
    #netG: AdaFill

    # lightweight_gan (2021)
    #netG: lightweight_gan
    #image_size: 512
    #latent_dim: 256
    #fmap_max: 512
    #fmap_inverse_coef: 12
    #transparent: False
    #greyscale: False
    #freq_chan_attn: False

    # CTSDG (2021)
    #netG: CTSDG

    # lama (2022) (no AMP)
    #netG: lama

    # MST (2021)
    #netG: MST

    # mat (2022) (compiling)
    #netG: mat
    #z_dim: 512
    #c_dim: 0
    #w_dim: 512
    #img_resolution: 512
    #img_channels: 3
    #noise_mode: const # const | random

    # ----Interpolation Generators----
    # cain (2020)
    #netG: CAIN
    #depth: 3
    #conv: WSConv # doconv | conv2d | gated | TBC | dynamic | MBConv | Involution | CondConv | fft | WSConv
    # Warning: Configure OutlookAttention dimension according to resolution manually (160 for 720p)
    #attention: CA # CA | OutlookAttention | A2Atttention | CBAM | CoTAttention | CoordAttention | ECAAttention | HaloAttention | ParNetAttention | TripletAttention | SKAttention | SGE | SEAttention | PolarizedSelfAttention
    #RG: 2 # ResidualGroup amount
    ## for dynamic
    #nof_kernels: 4
    #reduce: 4

    # rife 4.0
    #netG: rife
    #conv: conv2d # doconv | conv2d | gated | TBC | dynamic | MBConv | fft | WSConv
    ## for dynamic
    #nof_kernels: 4
    #reduce: 4

    # RRIN (2020)
    #netG: RRIN

    # ABME (2021)
    #netG: ABME

    # EDSC (2021) (2^x image size)
    # pip install cupy
    #netG: EDSC

    # sepconv enhanced (2021) (needs cupy)
    #netG: sepconv_enhanced

    # sepconv realtime (enet) (2022) (needs cupy)
    #netG: sepconv_rt
    #real_time: True
    #device: cuda
    #in_channels: 64
    #out_channels: 51

    # AdaCoFNet (compressed) / CDFI (2021) (needs cupy)
    #netG: CDFI

    #-----------Misc---------------
    # Restormer (2021) (1x model)
    #netG: restormer
    #inp_channels: 3
    #out_channels: 3
    #dim: 48
    #num_blocks: [4,6,6,8]
    #num_refinement_blocks: 4
    #heads: [1,2,4,8]
    #ffn_expansion_factor: 2.66
    #bias: False
    #LayerNorm_type: 'WithBias' # 'WithBias' | 'BiasFree'

# Discriminator options:
network_D:
    discriminator_criterion: MSE # MSE

    d_loss_fool_weight: 1 # inside the generator loop, trying to fool the disciminator
    d_loss_weight: 1 # inside own discriminator update
    
    # needs "pip3 install git+https://github.com/vballoli/nfnets-pytorch"
    WSConv_replace: True

    #netD: # in case there is no discriminator, leave it empty

    # VGG
    #netD: VGG
    #size: 256
    #in_nc: 3 #3
    #base_nf: 64
    #norm_type: 'batch'
    #act_type: 'leakyrelu'
    #mode: 'CNA'
    #convtype: 'Conv2D'
    #arch: 'ESRGAN'

    # VGG fea
    #netD: VGG_fea
    #size: 256
    #in_nc: 3
    #base_nf: 64
    #norm_type: 'batch'
    #act_type: 'leakyrelu'
    #mode: 'CNA'
    #convtype: 'Conv2D'
    #arch: 'ESRGAN'
    #spectral_norm: False
    #self_attention: False
    #max_pool: False
    #poolsize: 4


    #netD: VGG_128_SN

    # VGGFeatureExtractor
    #netD: VGGFeatureExtractor
    #feature_layer: 34
    #use_bn: False
    #use_input_norm: True
    #device: 'cpu'
    #z_norm: False

    # PatchGAN
    #netD: NLayerDiscriminator
    #input_nc: 3
    #ndf: 64
    #n_layers: 3
    #norm_layer: nn.BatchNorm2d
    #use_sigmoid: False
    #getIntermFeat: False
    #patch: True
    #use_spectral_norm: False

    # Multiscale
    #netD: MultiscaleDiscriminator
    #input_nc: 3
    #ndf: 64
    #n_layers: 3
    #norm_layer: nn.BatchNorm2d
    #use_sigmoid: False
    #num_D: 3
    #get_feats: False

    #netD: ResNet101FeatureExtractor
    #use_input_norm: True
    #device: 'cpu'
    #z_norm: False

    # MINC
    #netD: MINCNet

    # Pixel
    #netD: PixelDiscriminator
    #input_nc: 3
    #ndf: 64
    #norm_layer: nn.BatchNorm2d

    # EfficientNet (3-channel input)
    #netD: EfficientNet
    #EfficientNet_pretrain: 'efficientnet-b0'
    #num_classes: 1 # should be 1

    # ResNeSt (not working)
    #netD: ResNeSt
    #ResNeSt_pretrain: 'resnest50' # ["resnest50", "resnest101", "resnest200", "resnest269"]
    #pretrained: False # cant be true currently
    #num_classes: 1

    # Transformer (not working)
    #netD: TranformerDiscriminator
    #img_size: 256
    #patch_size: 1
    #in_chans: 3
    #num_classes: 1
    #embed_dim: 64
    #depth: 7
    #num_heads: 4
    #mlp_ratio: 4.
    #qkv_bias: False
    #qk_scale: None
    #drop_rate: 0.
    #attn_drop_rate: 0.
    #drop_path_rate: 0.
    #hybrid_backbone: None
    #norm_layer: 

    # context_encoder (num_classes can't be set, broadcasting warning will be shown, training works, but I am not sure if it will work correctly)
    #netD: context_encoder

    # Transformer (doesn't do init)
    #netD: ViT
    #image_size: 256
    #patch_size: 32
    #num_classes: 1
    #dim: 1024
    #depth: 6
    #heads: 16
    #mlp_dim: 2048
    #dropout: 0.1
    #emb_dropout: 0.1

    # Transformer (doesn't do init)
    #netD: DeepViT
    #image_size: 256
    #patch_size: 32
    #num_classes: 1
    #dim: 1024
    #depth: 6
    #heads: 16
    #mlp_dim: 2048
    #dropout: 0.1
    #emb_dropout: 0.1

    # RepVGG
    #netD: RepVGG
    #RepVGG_arch: RepVGG-A0 # RepVGG-A0, RepVGG-A1, RepVGG-A2, RepVGG-B0, RepVGG-B1, RepVGG-B1g2, RepVGG-B1g4, , RepVGG-B2, RepVGG-B2g2, RepVGG-B2g4, RepVGG-B3, RepVGG-B3g2, RepVGG-B3g4
    #num_classes: 1

    # squeezenet
    #netD: squeezenet
    #version: "1_1" # 1_0, 1_1
    #num_classes: 1

    # SwinTransformer (doesn't do init)
    #netD: SwinTransformer
    #hidden_dim: 96
    #layers: [2, 2, 6, 2]
    #heads: [3, 6, 12, 24]
    #channels: 3
    #num_classes: 1
    #head_dim: 32
    #window_size: 8
    #downscaling_factors: [4, 2, 2, 2]
    #relative_pos_embedding: True

    # mobilenetV3 (doesn't do init)
    #netD: mobilenetV3
    #mode: small # small, large
    #n_class: 1
    #input_size: 256

    # resnet
    #netD: resnet
    #resnet_arch: resnet50 # resnet50, resnet101, resnet152
    #num_classes: 1
    #pretrain: True
  
    # NFNet
    #netD: NFNet
    #num_classes: 1
    #variant: 'F0'         # F0 - F7
    #stochdepth_rate: 0.25 # 0-1, the probability that a layer is dropped during one step
    #alpha: 0.2            # Scaling factor at the end of each block
    #se_ratio: 0.5         # Squeeze-Excite expansion ratio
    #activation: 'gelu'    # or 'relu'

    # lvvit (2021)
    # Warning: Needs 'pip install timm==0.4.5'
    #netD: lvvit
    #img_size: 224
    #patch_size: 16
    #in_chans: 3
    #num_classes: 1
    #embed_dim: 768
    #depth: 12
    #num_heads: 12
    #mlp_ratio: 4.
    #qkv_bias: False
    #qk_scale: # None
    #drop_rate: 0.
    #attn_drop_rate: 0.
    #drop_path_rate: 0.
    #drop_path_decay: 'linear'
    #hybrid_backbone: # None
    ##norm_layer: nn.LayerNorm # Deafault: nn.LayerNorm / can't be configured
    #p_emb: '4_2'
    #head_dim: # None
    #skip_lam: 1.0
    #order: # None
    #mix_token: False
    #return_dense: False

    # timm
    # pip install timm
    # you can loop up models here: https://rwightman.github.io/pytorch-image-models/
    #netD: timm
    #timm_model: "tf_efficientnetv2_b0"

    #netD: resnet3d
    #model_depth: 50 # [10, 18, 34, 50, 101, 152, 200]

    # from lama (2021)
    #netD: FFCNLayerDiscriminator
    #FFCN_feature_weight: 1

    #netD: effV2
    #conv: fft # fft | conv2d
    #size: s # s | m | l | xl

    # x-transformers
    # pip install x-transformers
    #netD: x_transformers
    #image_size: 512
    #patch_size: 32
    #dim: 512
    #depth: 6
    #heads: 8

    #netD: mobilevit
    #size: xxs # xxs | xs | s

    # because of too many parameters, a seperate config file named "hrt_config.yaml" is available
    #netD: hrt

    # Attention U-Net (from asergan) (2021)
    netD: attention_unet
    num_in_ch: 3
    num_feat: 64
    skip_connection: True

    # U-Net (from RealESRGAN) (2021)
    #netD: unet
    #num_in_ch: 3
    #num_feat: 64
    #skip_connection: True

train: 
    # Adam8bit needs "pip install bitsandbytes-cuda111"
    # SGD_AGC needs "pip3 install git+https://github.com/vballoli/nfnets-pytorch"
    scheduler: AdamP # Adam, AdamP, Adam, SGDP, MADGRAD, Adam8bit, SGD_AGC, cosangulargrad [maybe broken], tanangulargrad [maybe broken]
    # ACG needs "pip3 install git+https://github.com/vballoli/nfnets-pytorch"
    AGC: False # currently only for generator, only works with certain optimizers
    lr_g: 0.0001 # 0.00005
    lr_d: 0.0001
    
    # AdamP, AGDP, MADGRAD, SGD_AGC, cosangulargrad, tanangulargrad
    weight_decay: 0.01

    # SGDP, MAGDRAD
    momentum: 0.9

    # AdamP, cosangulargrad, tanangulargrad
    betas0: 0.9
    betas1: 0.999
    
    # SGDP
    nesterov: True

    # MADGRAD, SGD_AGC, cosangulargrad, tanangulargrad
    eps: 1e-6

    ############################

    # Losses:
    L1Loss_weight: 0

    # HFENLoss
    HFEN_weight: 0
    loss_f: L1CosineSim # L1Loss | L1CosineSim
    kernel: 'log'
    kernel_size: 15
    sigma: 2.5
    norm: False

    # Elastic
    Elastic_weight: 0
    a: 0.2
    reduction_elastic: 'mean'

    # Relative L1
    Relative_l1_weight: 0
    l1_eps: .01
    reduction_relative: 'mean'

    # L1CosineSim (3-channel input)
    L1CosineSim_weight: 1
    loss_lambda: 5
    reduction_L1CosineSim: 'mean'

    # ClipL1
    ClipL1_weight: 0
    clip_min: 0.0
    clip_max: 10.0

    # FFTLoss
    FFTLoss_weight: 0
    loss_f_fft: L1Loss
    reduction_fft: 'mean'

    OFLoss_weight: 0

    # GPLoss
    GPLoss_weight: 0
    gp_trace: False
    gp_spl_denorm: False

    # CPLoss
    CPLoss_weight: 0
    rgb: True
    yuv: True
    yuvgrad: True
    cp_trace: False
    cp_spl_denorm: False
    yuv_denorm: False

    # TVLoss
    TVLoss_weight: 0
    tv_type: 'tv'
    p: 1

    # Contextual_Loss (3-channel input)
    Contextual_weight: 0.5
    crop_quarter: False
    max_1d_size: 100
    distance_type: 'cosine' # ["11", "l2", "consine"]
    b: 1.0
    band_width: 0.5
    # for vgg
    use_vgg: False
    net_contextual: 'vgg19'
    layers_weights: {'conv_1_1': 1.0, 'conv_3_2': 1.0}
    # for timm
    use_timm: True
    timm_model: "tf_efficientnetv2_b0"
    # for both
    calc_type: 'regular' # ["bilateral" | "symetric" | None]

    # Style (3-channel input)
    StyleLoss_weight: 0

    # PerceptualLoss
    perceptual_weight: 1
    net: PNetLin # PNetLin, DSSIM (?)
    pnet_type: 'vgg' # alex, squeeze, vgg
    pnet_rand: False
    pnet_tune: False
    use_dropout: True
    spatial: False
    version: '0.1' # only version
    lpips: True
    force_fp16_perceptual: False # not supported for cpu
    # you need to have tensorrt and torch_tensorrt, use docker or install it manually
    # adjust optimum model compile values in CustomTrainClass
    # fp16 will only work on fp16 compatible hardware, convert will only work with a GPU, FP16 untested
    perceptual_tensorrt: False 


    # high receptive field (HRF) perceptual loss
    # you can download it manually with this command, but the code will download it automatically if you don't have it
    # wget -P /content/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth
    hrf_perceptual_weight: 0
    force_fp16_hrf: True # not supported for cpu

    ColorLoss_weight: 0 # converts rgb to yuv and calculates l1, expected input is rgb
    FrobeniusNormLoss_weight: 0
    GradientLoss_weight: 0
    MultiscalePixelLoss_weight: 0
    SPLoss_weight: 0
    FFLoss_weight: 0

    # only if the network outputs 2 images, will use l1
    stage1_weight: 0 

    Lap_weight: 0

    # pytorch loss functions
    MSE_weight: 0
    BCE_weight: 0
    Huber_weight: 0
    SmoothL1_weight: 0

    # piq loss
    SSIMLoss_weight: 0
    MultiScaleSSIMLoss_weight: 0
    VIFLoss_weight: 0
    FSIMLoss_weight: 0 # unstable
    GMSDLoss_weight: 0 # unstable
    MultiScaleGMSDLoss_weight: 0 # unstable
    VSILoss_weight: 0 # unstable
    HaarPSILoss_weight: 0
    MDSILoss_weight: 0 # unstable
    BRISQUELoss_weight: 0 # unstable
    PieAPP_weight: 0
    DISTS_weight: 0
    # the losses below require a GPU
    # may be changed in the future, to have custom feature extractors
    IS_weight: 0 # unstable
    FID_weight: 0
    KID_weight: 0
    PR_weight: 0 # only fp32
    # todo(?): LPIPS
    force_piq_fp16: True

    # loss for CTSDG
    CTSDG_edge_weight: 0 #0.01
    CTSDG_projected_weight: 0 #0.1

    # loss for rife
    SOBEL_weight: 0 # not recommended, leave it at 0

    # Three different augmentation options, diffaug and muaraugment apply to both training loops,
    # while batch_aug only applies to the discriminator training loop
    augmentation_method: batch_aug # diffaug, MuarAugment, batch_aug, None
    # diffaug
    policy: 'color,translation,cutout'
    # MuarAugment
    N_TFMS: 3 # Number of transformations
    MAGN: 3 # Magnitude of augmentation applied
    N_COMPS: 4 # Number of compositions placed on each image
    N_SELECTED: 2 # Number of selected compositions for each image
    # batch_aug
    mixopts: ["blend", "rgb", "mixup", "cutmix", "cutmixup", "cutout"]
    mixprob: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]  # , 1.0]
    mixalpha: [0.6, 1.0, 1.2, 0.7, 0.7, 0.001]  # , 0.7]
    aux_mixprob: 1.0
    aux_mixalpha: 1.2

    # Metrics
    metrics: [] # PSNR | SSIM | AE | MSE

In [ ]:
#@title aug_config.yaml
%%writefile /content/Colab-traiNNer/code/aug_config.yaml 
# Settings to configure OTF image augmentations for lrhr dataloader
# p is always the probability of applying the augmentation
# kernel_size must always be odd
# sigma variables are standard deviation
# Leave variable blank if setting to None
  ColorJitter:
    p: 0.5
    brightness: 0
    contrast: 0
    saturation: 0
    hue: 0
  RandomGaussianNoise:
    p: 0.5
    mean: 0.0
    var_limit: [10.0, 50.0]  # Variance range; 0-255 for sigma_calc 'sig', squared for 'var'
    prob_color: 0.5  # Probability of color rather than grayscale noise
    multi: True
    mode: 'gauss'  # ['gauss' | 'speckle]
    sigma_calc: 'sig'  # ['sig' | 'var']
  RandomPoissonNoise:
    p: 0.5
    prob_color: 0.5  # Probability of color rather than grayscale noise
    scale_range: [0.5, 1.0]  # Range for random selection noise scale
  RandomSPNoise:
    p: 0.5
    prob: 0.1  # Threshold to control level of noise
  RandomSpeckleNoise:  # Just Gaussian noise set to speckle
    p: 0.5
    mean: 0.0
    var_limit: [0.04, 0.12]
    prob_color: 0.5
    sigma_calc: 'var'
  RandomCompression:
    p: 0.5
    min_quality: 20
    max_quality: 90
    compression_type: '.jpg'  # ['.jpg' | '.webp']
  RandomAverageBlur:
    p: 0.5
    kernel_size: 3  # Maximum kernel size (must be >=3)
  RandomBilateralBlur:
    p: 0.5
    kernel_size: 3  # Maximum kernel size (must be >=3)
    sigmaX: 0.5
    sigmaY: 0.5
  RandomBoxBlur:
    p: 0.5
    kernel_size: 3  # Maximum kernel size (must be >=3)
  RandomGaussianBlur:
    p: 0.5
    kernel_size: 3  # Maximum kernel size (must be >=3)
    sigmaX: 0.5
    sigmaY: 0.5
  RandomMedianBlur:
    p: 0.5
    kernel_size: 3  # Maximum kernel size (must be >=3)
  RandomMotionBlur:
    p: 0.5
    kernel_size: 3  # Maximum kernel size (must be >=3)
    per_channel: False  # Apply motion blur simultaneously or individually per channel
  RandomComplexMotionBlur:
    p: 0.5
    size: [100, 100]
    complexity: 0  # Modifies length and variance of motion blur path (may need to be range 0-1?)
    eps: 0.1  # Small error for numerical stability
  RandomAnIsoBlur:
    p: 0.5
    min_kernel_size: 1
    kernel_size: 3  # Maximum kernel size (must be >=3)
    sigmaX: 0.5
    sigmaY: 0.5
    angle:  # Rotation angle for anisotropic filters
    noise:  # Multiplicative kernel noise
    scale: 1  # To prevent filter misalignment (i.e. with nearest neighbor); scale 1 does not shift pixels
  RandomSincBlur:
    p: 0.5
    min_kernel_size: 7
    kernel_size: 21  # Maximum kernel size (must be >=3)
    min_cutoff:  # Minimum omega cutoff frequency in radians (max: pi)
  BayerDitherNoise:
    p: 0.5
  FSDitherNoise:
    p: 0.5
  FilterMaxRGB:
    p: 0.5
  FilterColorBalance:
    p: 0.5
    percent: 1  # Amount of balance to apply
    random_params: False  # If true, randomizes percent from 0 to percent
  FilterUnsharp:
    p: 0.5
    blur_algo: 'median'  # ['median' | None]; only used for 'laplacian'
    kernel_size:  # Leave blank to select randomly from [1, 3, 5]
    strength: 0.3  # Strength of filter applied (range 0-1)
    unsharp_algo: 'laplacian'  # ['DoG' | 'laplacian]
  FilterCanny:
    p: 0.5
    sigma: 0.33
    bin_thresh: False  # Flag to apply binarize operation
    threshold: 127  # Cutoff value for binarize (0-255)
  SimpleQuantize:
    p: 0.5
    rgb_range: 40  # Higher values increase value range contained in each bin (1-255)
  KMeansQuantize:
    p: 0.5
    n_colors: 128  # Number of colors in quantized image (1-255)
  CLAHE:  # Contrast-Limited Adaptive Histogram Equalization
    p: 0.5
    clip_limit: 4.0  # Upper threshold value for contrast limiting (min 1)
    tile_grid_size: [8, 8]
  RandomGamma:
    p: 0.5
    gamma_range: [80, 120]  #  Range to randomly select gamma from
    gain: 1  # Constant multiplier for gamma adjustment
  Superpixels:
    p: 0.5
    p_replace: 0.1  # Probability for any segment of pixels within being replaced by their aggregate color
    n_segments: 100  # Approximate target of number of superpixels to generate
    cs:  # Colorspace conversion; ['lab' | 'hsv' | None]
    algo: 'slic'  # Superpixels algorithm; ['seeds' | 'slic' | 'slico' | 'mslic' | 'sk_slic' | 'sk_felzenszwalb']
    n_iters: 10  # Only applies for certain algorithms
    kind: 'mix'  # How to aggregate colors; ['avg' | 'median' | 'mix']
    reduction:  # Post-process segment to reduce colors for algos that produce more than n_segments (sk_felzenszwalb); ['selective' | 'cluster' | 'rag']
    max_size: 128  # Max image size at which augmentation applied, will be downscaled if necessary [int or None]
    interpolation: 'BILINEAR'  # ['NEAREST' | 'BILINEAR' | 'AREA' | 'BICUBIC' | 'LANCZOS']
  RandomCameraNoise:
    p: 0.5
    demosaic_fn: 'malvar'  # ['malvar' | 'pixelshuffle' | 'menon' | 'bilinear']
    xyz_arr: 'D50'  # Matrix to use for RGB to XYZ conversion; ['D50 | 'D65']
    rg_range: [1.2, 2.4]  # Red gain range for white balance
    bg_range: [1.2, 2.4]  # Blue gain range for white balance
    random_params: False  # Initialize with random parameters if True

In [ ]:
#@title realesrgan_aug_config.yaml
%%writefile /content/Colab-traiNNer/code/realesrgan_aug_config.yaml 
# https://github.com/xinntao/Real-ESRGAN/blob/35ee6f781e9a5a80d5f2f1efb9102c9899a81ae1/options/train_realesrgan_x2plus.yml
blur_kernel_size: 21
kernel_list: ['iso', 'aniso', 'generalized_iso', 'generalized_aniso', 'plateau_iso', 'plateau_aniso']
kernel_prob: [0.45, 0.25, 0.12, 0.03, 0.12, 0.03]
sinc_prob: 0.1
blur_sigma: [0.2, 3]
betag_range: [0.5, 4]
betap_range: [1, 2]

blur_kernel_size2: 21
kernel_list2: ['iso', 'aniso', 'generalized_iso', 'generalized_aniso', 'plateau_iso', 'plateau_aniso']
kernel_prob2: [0.45, 0.25, 0.12, 0.03, 0.12, 0.03]
sinc_prob2: 0.1
blur_sigma2: [0.2, 1.5]
betag_range2: [0.5, 4]
betap_range2: [1, 2]

final_sinc_prob: 0.8

use_hflip: True
use_rot: False

# the first degradation process
resize_prob: [0.2, 0.7, 0.1]  # up, down, keep
resize_range: [0.15, 1.5]
gaussian_noise_prob: 0.5
noise_range: [1, 30]
poisson_scale_range: [0.05, 3]
gray_noise_prob: 0.4
jpeg_range: [30, 95]

# the second degradation process
second_blur_prob: 0.8
resize_prob2: [0.3, 0.4, 0.3]  # up, down, keep
resize_range2: [0.3, 1.2]
gaussian_noise_prob2: 0.5
noise_range2: [1, 25]
poisson_scale_range2: [0.05, 2.5]
gray_noise_prob2: 0.4
jpeg_range2: [30, 95]

## Run Training

In [ ]:
%cd '/content/Colab-traiNNer/code'
!python train.py

# Creating Kernels

Be aware that this takes a long time

In [ ]:
%cd /content
!git clone https://github.com/victorca25/DLIP
%cd /content/DLIP/kgan/

In [ ]:
# only works with GPU
!python train.py --input-dir /content/hr --X4 --output-dir /content/kernels

# Misc (Data)

In [ ]:
#@title Resize folder
from tqdm import tqdm
import os
import glob
import cv2
import threading
import shutil
import hashlib
import PIL
from PIL import Image
import numpy as np
import hashlib

rootdir = "/content/val_lr/"
destination = "/content/val_lr/"
broken_folder = "/content/"

resize_method = 'PIL' #@param ["OpenCV", "PIL"] {allow-input: false}

files = glob.glob(rootdir + '/**/*.png', recursive=True)
files_jpg = glob.glob(rootdir + '/**/*.jpg', recursive=True)
files_jpeg = glob.glob(rootdir + '/**/*.jpeg', recursive=True)
files_webp = glob.glob(rootdir + '/**/*.webp', recursive=True)
files.extend(files_jpg)
files.extend(files_jpeg)
files.extend(files_webp)
err_files=[]

image_size = 64

for file in tqdm(files):
    image = cv2.imread(file)
    if image is not None:
        #####################################
        # resize with opencv
        if resize_method == "OpenCV":
          resized = cv2.resize(image, (image_size,image_size), interpolation=cv2.INTER_AREA)

        # resize with PIL
        elif resize_method == "PIL":
          image = Image.fromarray(image)
          image = image.resize((image_size,image_size))
          resized = np.asarray(image)
        #####################################

        hash_md5 = hashlib.md5()
        with open(file, "rb") as f:
          for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)

        cv2.imwrite(file, resized)

In [ ]:
#@title creating tiled images (image grids) (with skip)
import cv2
import numpy
import glob
import shutil
import tqdm
import os
import PIL
from PIL import Image
import numpy as np

resize_method = 'PIL' #@param ["OpenCV", "PIL"] {allow-input: false}
grayscale = False #@param {type:"boolean"}

rootdir = '/content/' #@param {type:"string"}
destination_dir = "/content/" #@param {type:"string"}
broken_dir = '/content/opencv_fail/' #@param {type:"string"}
 
files = glob.glob(rootdir + '/**/*.png', recursive=True)
files_jpg = glob.glob(rootdir + '/**/*.jpg', recursive=True)
files_jpeg = glob.glob(rootdir + '/**/*.jpeg', recursive=True)
files_webp = glob.glob(rootdir + '/**/*.webp', recursive=True)
files.extend(files_jpg)
files.extend(files_jpeg)
files.extend(files_webp)
err_files=[]

amount_tiles = 8 #@param
image_size = 256 #@param

filepos = 0
img_cnt = 0
filename_cnt = 0

if grayscale == True:
  tmp_img = numpy.zeros((amount_tiles*image_size,amount_tiles*image_size))
elif grayscale == False:
  tmp_img = numpy.zeros((amount_tiles*image_size,amount_tiles*image_size, 3))

with tqdm.tqdm(files) as pbar:
  while True:
      if grayscale == True:
        image = cv2.imread(files[filepos], cv2.IMREAD_GRAYSCALE)
      elif grayscale == False:
        image = cv2.imread(files[filepos])

      filepos += 1

      if image is not None:
        
        i = img_cnt % amount_tiles
        j = img_cnt // amount_tiles

        #####################################
        # resize with opencv
        if resize_method == "OpenCV":
          image = cv2.resize(image, (image_size,image_size), interpolation=cv2.INTER_AREA)

        # resize with PIL
        elif resize_method == "PIL":
          if grayscale == True:
            image = Image.fromarray(image)
            image = image.resize((image_size,image_size))
            image = np.asarray(image)
          if grayscale == False:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)
            image = image.resize((image_size,image_size), resample=PIL.Image.LANCZOS)
            image = np.asarray(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #####################################

        tmp_img[i*image_size:(i+1)*image_size, j*image_size:(j+1)*image_size] = image
        img_cnt += 1
      else:
        print(files[filepos])
        print(f'{broken_dir}/{os.path.basename(files[filepos])}')
        shutil.move(files[filepos], f'{broken_dir}/{os.path.basename(files[filepos])}')

      if img_cnt == (amount_tiles*amount_tiles):
        #cv2.imwrite(destination_dir+str(filename_cnt)+".jpg", tmp_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        cv2.imwrite(destination_dir+str(filename_cnt)+".webp", tmp_img)
        filename_cnt += 1
        img_cnt = 0
      pbar.update(1)

In [ ]:
#@title convert to onnx
#@markdown Make sure the input dimensions are correct. Maybe a runtime restart is needed if it complains about ``TypeError: forward() missing 1 required positional argument``. Make sure you only run the required cells.
from torch.autograd import Variable
model = CustomTrainClass()
checkpoint_path = '/content/Checkpoint_0_0.ckpt' #@param
output_path = '/content/output.onnx' #@param
model = model.load_from_checkpoint(checkpoint_path) # start training from checkpoint, warning: apperantly global_step will be reset to zero and overwriting validation images, you could manually make an offset
dummy_input = Variable(torch.randn(1, 1, 64, 64))

model.to_onnx(output_path, input_sample=dummy_input)

In [ ]:
#@title copy pasting data to create artificatial dataset for debugging
import shutil
from random import random
from tqdm import tqdm
for i in tqdm(range(5000)):
  shutil.copy("/content/4k/0.jpg", "/content/4k/"+str(random())+"jpg")

In [ ]:
#@title pip list with space
!pip list | tail -n +3 | awk '{print $1}' | xargs pip show | grep -E 'Location:|Name:' | cut -d ' ' -f 2 | paste -d ' ' - - | awk '{print $2 "/" tolower($1)}' | xargs du -sh 2> /dev/null | sort -hr

In [ ]:
#@title tiling script
import cv2
import numpy
import glob
import shutil
import tqdm
import os
from multiprocessing.pool import ThreadPool as ThreadPool

rootdir = '/content/' #@param {type:"string"}
destination_dir = "/content/" #@param {type:"string"}
broken_dir = '/content/' #@param {type:"string"}
threads = 2 #@param
tile_size = 256 #@param

files = glob.glob(rootdir + '/**/*.png', recursive=True)
files_jpg = glob.glob(rootdir + '/**/*.jpg', recursive=True)
files_jpeg = glob.glob(rootdir + '/**/*.jpeg', recursive=True)
files_webp = glob.glob(rootdir + '/**/*.webp', recursive=True)
files.extend(files_jpg)
files.extend(files_jpeg)
files.extend(files_webp)
err_files=[]

pool = ThreadPool(threads)

def tiling(f):
  image = cv2.imread(f)
  if image is not None:
      counter = 0

      x = image.shape[0]
      y = image.shape[1]

      x_amount = x // tile_size
      y_amount = y // tile_size

      for i in range(x_amount):
        for j in range(y_amount):
          crop = image[i*tile_size:(i+1)*tile_size, (j*tile_size):(j+1)*tile_size]
          cv2.imwrite(os.path.join(destination_dir, os.path.splitext(os.path.basename(f))[0] + str(counter) + ".png"), crop)
          counter += 1

    else:
        print(f'Broken file: {os.path.basename(f)}')
        shutil.move(f, f'{broken_dir}/{os.path.basename(f)}')

        
pool.map(tiling, files)

In [ ]:
#@title create landmarks (for DFDNet)
!pip install face-alignment
!pip install matplotlib --upgrade

import face_alignment
from skimage import io
import numpy as np
import glob
from tqdm import tqdm
import os
import shutil

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)

unchecked_input_path = '/content/ffhq' #@param {type:"string"}
checked_output_path = '/content/ffhq' #@param {type:"string"}
failed_output_path = '/content/ffhq' #@param {type:"string"}
landmark_output_path = '/content/landmarks' #@param {type:"string"}

if not os.path.exists(unchecked_input_path):
    os.makedirs(unchecked_input_path)
if not os.path.exists(checked_output_path):
    os.makedirs(checked_output_path)
if not os.path.exists(failed_output_path):
    os.makedirs(failed_output_path)
if not os.path.exists(landmark_output_path):
    os.makedirs(landmark_output_path)

files = glob.glob(unchecked_input_path + '/**/*.png', recursive=True)
files_jpg = glob.glob(unchecked_input_path + '/**/*.jpg', recursive=True)
files.extend(files_jpg)
err_files=[]

for f in tqdm(files):
  input = io.imread(f)
  preds = fa.get_landmarks(input)
  if preds is not None:
    np.savetxt(os.path.join(landmark_output_path, os.path.basename(f)+".txt"), preds[0], delimiter=' ', fmt='%1.3f')   # X is an array
    shutil.move(f, os.path.join(checked_output_path,os.path.basename(f)))
  else:
    shutil.move(f, os.path.join(failed_output_path,os.path.basename(f)))

In [ ]:
#@title download DictionaryCenter512 for DFDNet
!mkdir /content/DictionaryCenter512
%cd /content/DictionaryCenter512
!gdown --id 1sEB9j3s7Wj9aqPai1NF-MR7B-c0zfTin
!gdown --id 1H4kByBiVmZuS9TbrWUR5uSNY770Goid6
!gdown --id 10ctK3d9znZ9nGN3d1Z77xW3GGshbeKBb
!gdown --id 1gcwmrIZjPFVu-cHjdQD6P4luohkPsil-
!gdown --id 1rJ8cORPxbJsIVAiNrjBag0ihaY_Mvurn
!gdown --id 1LkfJv2a3ud-mefAc1eZMJuINuNdSYgYO
!gdown --id 1LH-nxD__icSJvTiAbXAXDch03oDtbpkZ
!gdown --id 1JRTStLFsQ8dwaQjQ8qG5fNyrOvo6Tcvd
!gdown --id 1Z4AkU1pOYTYpdbfljCgNMmPilhdEd0Kl
!gdown --id 1Z4e1ltB3ACbYKzkoMBuVtzZ7a310G4xc
!gdown --id 1fqWmi6-8ZQzUtZTp9UH4hyom7n4nl8aZ
!gdown --id 1wfHtsExLvSgfH_EWtCPjTF5xsw3YyvjC
!gdown --id 1Jr3Luf6tmcdKANcSLzvt0sjXr0QUIQ2g
!gdown --id 1sPd4_IMYgqGLol0gqhHjBedKKxFAxswR
!gdown --id 1eVFjXJRnBH4mx7ZbAmZRwVXZNUbgCQec
!gdown --id 1w0GfO_KY775ZVF3KMk74ya6QL_bNU4cJ

#!mkdir /content/DFDNet/weights/
#%cd /content/DFDNet/weights/
#!gdown --id 1SfKKZJduOGhDD27Xl01yDx0-YSEkL2Aa

In [ ]:
#@title getting ffhq test data
%cd /content/
!gdown --id 1VE5tnOKcfL6MoV839IVCCw5FhJxIgml5
!7z x data.zip

# Download pretrain

In [ ]:
#@title getting DFDNet pretrain
%cd /content
!gdown --id 1UCo7YEbLLa1_87b0AoWmzhTGyrw-26nb

In [ ]:
#@title downloading places2 dfnet
%cd /content/
!gdown --id 1SGJ_Z9kpchdnZ3Qwwf4HnN-Cq-AeK7vH # dfnet places2

In [ ]:
#@title download rfr paris model and fix state_dict
# rfr paris
%cd /content/
!gdown --id 1jnUb-EvBw9DcwyWUQyWDdN9o42BPH7uT

#https://discuss.pytorch.org/t/dataparallel-changes-parameter-names-issue-with-load-state-dict/60211
import torch
from collections import OrderedDict
state_dict = torch.load("/content/checkpoint_paris.pth", map_location='cpu')
new_state_dict = OrderedDict()

for k, v in state_dict['generator'].items():
  if k == 'Pconv1.weight':
      name = 'conv1.weight'
  elif k == 'Pconv2.weight':
      name = 'conv2.weight'
  elif k == 'Pconv21.weight':
      name = 'conv21.weight'
  elif k == 'Pconv22.weight':
      name = 'conv22.weight'
  else:
      name = k

  new_state_dict[name] = v

torch.save(new_state_dict, '/content/converted.pth')

In [ ]:
#@title download and create fixed lama pretrain
!pip install omegaconf

%cd /content
!pip3 install wldhx.yadisk-direct
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip

import torch
from collections import OrderedDict
state_dict = torch.load("/content/big-lama/models/best.ckpt", map_location='cpu')
new_state_dict = OrderedDict()

for k, v in state_dict['state_dict'].items():
  name = k.replace("generator.", "")
  new_state_dict[name] = v

torch.save(new_state_dict, '/content/converted.pth')

# Misc

A summary of all interesting inpainting generators that are not trainable with my code.

--------------------------------------------------

``Broken generators:``

Generators that are not included here since I can't seem to make them work properly:

AOT-GAN (2021): [researchmm/AOT-GAN-for-Inpainting](https://github.com/researchmm/AOT-GAN-for-Inpainting)

    Couldn't get generator working.

PenNet [no AMP] (2019): [researchmm/PEN-Net-for-Inpainting](https://github.com/researchmm/PEN-Net-for-Inpainting/)

    Always outputs white for some reason.

CRA [no AMP] (2019): [wangyx240/High-Resolution-Image-Inpainting-GAN](https://github.com/wangyx240/High-Resolution-Image-Inpainting-GAN)

    Likes to create the color pink.

Global [no AMP] (2020): [SayedNadim/Global-and-Local-Attention-Based-Free-Form-Image-Inpainting](https://github.com/SayedNadim/Global-and-Local-Attention-Based-Free-Form-Image-Inpainting)

    Always outputs white for some reason.

crfill (2020): [zengxianyu/crfill](https://github.com/zengxianyu/crfill)

    No clear instructions/code result in broken results. Unreleased training code makes a correct implementation harder.

--------------------------------------------------

``Non-Pytorch generators:``

PSR-Net (2020): [sfwyly/PSR-Net](https://github.com/sfwyly/PSR-Net)

    Uses Tensorflow 2

co-mod-gan (2021): [zsyzzsoft/co-mod-gan](https://github.com/zsyzzsoft/co-mod-gan)

    Has a web demo and (a broken link to a) docker. Relies on Tensorflow 1.15 / StyleGAN2 code. A Colab by me for this can be found inside https://github.com/styler00dollar/Colab-co-mod-gan.

Diverse-Structure-Inpainting (2021): [USTC-JialunPeng/Diverse-Structure-Inpainting](https://github.com/USTC-JialunPeng/Diverse-Structure-Inpainting)

    Tensorflow 1

R-MNet (2021): [Jireh-Jam/R-MNet-Inpainting-keras](https://github.com/Jireh-Jam/R-MNet-Inpainting-keras)

    Not sure if there is much new and interesting stuff.

Hypergraphs (2021): [GouravWadhwa/Hypergraphs-Image-Inpainting](https://github.com/GouravWadhwa/Hypergraphs-Image-Inpainting)

    Uses custom conv layer (that is implemented with tensorflow). It sounds interesting, but I got errors when I tried to port it to pytorch.

PEPSI (2019): [Forty-lock/PEPSI-Fast_image_inpainting_with_parallel_decoding_network](https://github.com/Forty-lock/PEPSI-Fast_image_inpainting_with_parallel_decoding_network)

    The net dcpV2 uses.

Region (2019): [vickyFox/Region-wise-Inpainting](https://github.com/vickyFox/Region-wise-Inpainting)

--------------------------------------------------

``Pytorch generators that I never tested:``

SPL (2021): [WendongZh/SPL](https://github.com/WendongZh/SPL)

WTAM (2020): [ChenWang8750/WTAM](https://github.com/ChenWang8750/WTAM)

MPI (2020): [ChenWang8750/MPI-model](https://github.com/ChenWang8750/MPI-model)

Edge-LBAM (2021): [wds1998/Edge-LBAM](https://github.com/wds1998/Edge-LBAM)

VCNET (2020): [birdortyedi/vcnet-blind-image-inpainting](https://github.com/birdortyedi/vcnet-blind-image-inpainting)

    Blind image inpainting without masks.

DFMA (2020): [mprzewie/dmfa_inpainting](https://github.com/mprzewie/dmfa_inpainting)

GIN (2020): [rlct1/gin-sg](https://github.com/rlct1/gin-sg) and [rlct1/gin](https://github.com/rlct1/gin)

StructureFlow (2019): [RenYurui/StructureFlow](https://github.com/RenYurui/StructureFlow)

    Needs special files.

GMCNN (2018): [shepnerd/inpainting_gmcnn](https://github.com/shepnerd/inpainting_gmcnn)

    The net dcpV1 used iirc.

ShiftNet (2018): [Zhaoyi-Yan/Shift-Net_pytorch](https://github.com/Zhaoyi-Yan/Shift-Net_pytorch)

--------------------------------------------------
``Soon:``

ICT (2021): [raywzy/ICT](https://github.com/raywzy/ICT) (code released, waiting for pre-trained models)

MuFA-Net (2021): [ChenWang8750/MuFA-Net](https://github.com/ChenWang8750/MuFA-Net)

GCM-Net (2021): [ZhengHuanCS/GCM-Net](https://github.com/ZhengHuanCS/GCM-Net)

--------------------------------------------------

``No training code:``

SC-FEGAN (2019): [run-youngjoo/SC-FEGAN](https://github.com/run-youngjoo/SC-FEGAN)
